In [1]:
import pandas as pd

from hardware.opentrons.http_communications import OpentronsAPI

from hardware.cameras import PendantDropCamera

from hardware.sensor.sensor_api import SensorAPI
from hardware.opentrons.formulater import Formulater

from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter

from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration, get_well_id_solution
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model

from hardware.opentrons.droplet_manager import DropletManager


opentrons_api = OpentronsAPI()

opentrons_api.initialise()

config = Configuration(http_api=opentrons_api)

labware = config.load_labware()

containers = config.load_containers()

pipettes = config.load_pipettes()

left_pipette = pipettes["left"]

right_pipette = pipettes["right"]

pendant_drop_camera = PendantDropCamera()
sensor_api = SensorAPI()
plotter = Plotter()
settings = load_settings()
logger = Logger(
    name="protocol",
    file_path=f'experiments/{settings["EXPERIMENT_NAME"]}/meta_data',
)

formulater = Formulater(
    left_pipette=left_pipette,
    right_pipette=right_pipette,
    containers=containers,
    logger=logger
)

droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pendant_drop_camera,
    opentrons_api=opentrons_api,
    plotter=plotter,
    logger=logger

)

c:\Users\pimda\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-24 16:11:54,335 - INFO - Protocol already uploaded, using existing protocol.
2025-03-24 16:12:04,173 - INFO - Run created succesfully (ID: d45f4e64-f3d3-40c0-8fd1-613b92491ef5).
2025-03-24 16:12:06,882 - INFO - All custom labware definitions added.
2025-03-24 16:12:07,977 - INFO - Labware loaded successfully.
2025-03-24 16:12:08,017 - INFO - Containers loaded successfully.
2025-03-24 16:12:12,172 - INFO - Pipettes loaded successfully.


In [2]:
opentrons_api.home()

2025-03-24 16:12:39,264 - INFO - Robot homed.


In [3]:
drop_parameters = drop_parameters = {
    "drop_volume": 11,
    "max_measure_time": 30,
    "flow_rate": 1,
}
st_t, img_ana = droplet_manager.measure_pendant_drop(source=containers['7A1'], drop_parameters=drop_parameters)
# droplet_manager._make_pendant_drop(source=containers['7A1'], drop_volume=11, flow_rate=1, drop_count=1)

2025-03-24 16:12:47,331 - INFO - Picked up needle.
2025-03-24 16:12:47,331 - INFO - Start measurment of pendant drop of 7A1 with drop volume 11.0 uL and drop count 1.
2025-03-24 16:12:55,642 - INFO - Done with mixing in 7A1 with order before, with volume 15 uL, repeated 3 times
2025-03-24 16:12:57,229 - WARNING - Aspiration volume is larger than container volume!
2025-03-24 16:12:57,229 - INFO - Aspirated 17 uL from 7A1 with left pipette.
2025-03-24 16:13:01,160 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-24 16:13:05,547 - INFO - Tip/needle cleaned on sponge.
2025-03-24 16:13:10,726 - INFO - Air gap of 3 uL removed in left pipette.
2025-03-24 16:13:10,735 - INFO - Dispensing pendant drop.
2025-03-24 16:13:22,743 - INFO - Camera: start measuring 7A1.
2025-03-24 16:13:53,540 - INFO - Camera: stopped measurement
2025-03-24 16:13:54,505 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-24 16:13:58,203 - INFO - Container: dispensed 17 uL into this container from s

In [7]:
droplet_manager._return_pendant_drop(source=containers['7A1'], drop_volume=11)

2025-03-24 10:32:57,799 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-24 10:33:01,486 - INFO - Container: dispensed 17 uL into this container from source 7A1 containing n.a. mM empty.
2025-03-24 10:33:01,486 - INFO - Dispensed 17 uL into well 7A1 with left pipette.
2025-03-24 10:33:01,486 - INFO - Returned volume in tip to source.


In [17]:
import numpy as np
np.mean([item[1] for item in st_t])
# left_pipette.return_needle()

72.03158788782346

In [6]:
droplet_manager._make_pendant_drop(
    source=containers["7A1"],
    drop_volume=11,
    flow_rate=1,
    drop_count=1
)

2025-03-24 16:20:28,945 - INFO - Done with mixing in 7A1 with order before, with volume 15 uL, repeated 3 times
2025-03-24 16:20:30,531 - INFO - Container: aspirated 17 uL from this container with content n.a. mM empty.
2025-03-24 16:20:30,531 - INFO - Aspirated 17 uL from 7A1 with left pipette.
2025-03-24 16:20:34,463 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-24 16:20:38,833 - INFO - Tip/needle cleaned on sponge.
2025-03-24 16:20:44,009 - INFO - Air gap of 3 uL removed in left pipette.
2025-03-24 16:20:44,014 - INFO - Dispensing pendant drop.


In [8]:
droplet_manager._return_pendant_drop(
    source=containers['7A1'],
    drop_volume=11
)

2025-03-24 16:39:13,296 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-24 16:39:16,976 - INFO - Container: dispensed 17 uL into this container from source 7A1 containing n.a. mM empty.
2025-03-24 16:39:16,976 - INFO - Dispensed 17 uL into well 7A1 with left pipette.
2025-03-24 16:39:16,976 - INFO - Returned volume in tip to source.


In [7]:
pendant_drop_camera.stop_capture()
pendant_drop_camera.stop_stream()

2025-03-24 16:21:47,512 - INFO - Camera: stopped measurement


In [16]:
drop_parameters = drop_parameters = {
    "drop_volume": 11,
    "max_measure_time": 30,
    "flow_rate": 1,
}
st_t, img_ana = droplet_manager.measure_pendant_drop(
    source=containers["7A1"], drop_parameters=drop_parameters
)

2025-03-24 10:42:35,453 - INFO - Start measurment of pendant drop of 7A1 with drop volume 11.0 uL and drop count 1.
2025-03-24 10:42:44,799 - INFO - Done with mixing in 7A1 with order before, with volume 15 uL, repeated 3 times
2025-03-24 10:42:46,385 - INFO - Container: aspirated 17 uL from this container with content n.a. mM empty.
2025-03-24 10:42:46,385 - INFO - Aspirated 17 uL from 7A1 with left pipette.
2025-03-24 10:42:50,318 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-24 10:42:54,702 - INFO - Tip/needle cleaned on sponge.
2025-03-24 10:42:59,871 - INFO - Air gap of 3 uL removed in left pipette.
2025-03-24 10:42:59,882 - INFO - Dispensing pendant drop.
2025-03-24 10:43:11,881 - INFO - Camera: start measuring 7A1.
2025-03-24 10:43:42,668 - INFO - Camera: stopped measurement
2025-03-24 10:43:43,603 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-24 10:43:47,288 - INFO - Container: dispensed 17 uL into this container from source 7A1 containing n.a. mM e

In [4]:
formulater.wash(repeat=3)

2025-03-24 16:14:19,293 - INFO - Picked up tip.
2025-03-24 16:14:28,911 - INFO - Touched tip performed, repeated 1 times
2025-03-24 16:14:28,911 - INFO - Container: aspirated 300 uL from this container with content pure mM water.
2025-03-24 16:14:28,911 - INFO - Aspirated 300 uL from 6A1 with right pipette.
2025-03-24 16:14:36,526 - INFO - Touched tip performed, repeated 1 times
2025-03-24 16:14:36,526 - INFO - Container: dispensed 300 uL into this container from source 6A1 containing pure mM water.
2025-03-24 16:14:36,526 - INFO - Dispensed 300 uL into well 3A1 with right pipette.
2025-03-24 16:14:53,860 - INFO - Done with mixing in 3A1 with order after, with volume 20 uL, repeated 5 times
2025-03-24 16:15:04,276 - INFO - Touched tip performed, repeated 1 times
2025-03-24 16:15:04,276 - INFO - Container: aspirated 300 uL from this container with content pure mM water.
2025-03-24 16:15:04,291 - INFO - Aspirated 300 uL from 3A1 with right pipette.
2025-03-24 16:15:09,843 - INFO - Dispen

In [10]:
left_pipette.return_needle()

2025-03-24 16:40:11,644 - INFO - Returned needle.


In [9]:
left_pipette.clean_on_sponge()

2025-03-24 16:39:59,561 - INFO - Tip/needle cleaned on sponge.


In [10]:
offset = {
    'x': 0,
    'y': 0,
    'z': -23.4
}
left_pipette.move_to_well(container=containers['drop_stage'], offset=offset)

In [5]:
left_pipette.pick_up_tip()

2025-03-24 09:14:29,199 - INFO - Picked up tip.


In [2]:
opentrons_api.home()

2025-03-24 08:56:48,923 - INFO - Robot homed.


In [12]:
left_pipette.aspirate(
    volume=10,
    source = containers["3A1"],
    flow_rate=1
)

2025-03-24 09:34:48,362 - INFO - Container: aspirated 10 uL from this container with content pure mM water.
2025-03-24 09:34:48,363 - INFO - Aspirated 10 uL from 3A1 with left pipette.


In [13]:
left_pipette.dispense(volume=10, destination=containers["3A1"], flow_rate=1)

2025-03-24 09:35:07,800 - INFO - Container: dispensed 10 uL into this container from source 3A1 containing pure mM water.
2025-03-24 09:35:07,800 - INFO - Dispensed 10 uL into well 3A1 with left pipette.


In [4]:
well_id_water = get_well_id_solution(containers=containers, solution_name="water")
well_id_trash = get_well_id_solution(containers=containers, solution_name="trash")
if not right_pipette.has_tip:
    right_pipette.pick_up_tip()

right_pipette.aspirate(
    volume=300,
    source=containers[well_id_water],
    touch_tip=True
)
right_pipette.dispense(
    volume=300,
    destination=containers["3A1"],
    touch_tip=True
)

2025-03-24 09:39:52,732 - INFO - Picked up tip.
2025-03-24 09:40:02,299 - INFO - Touched tip performed, repeated 1 times
2025-03-24 09:40:02,299 - INFO - Container: aspirated 300 uL from this container with content pure mM water.
2025-03-24 09:40:02,299 - INFO - Aspirated 300 uL from 6A1 with right pipette.
2025-03-24 09:40:09,932 - INFO - Touched tip performed, repeated 1 times
2025-03-24 09:40:09,947 - INFO - Container: dispensed 300 uL into this container from source 6A1 containing pure mM water.
2025-03-24 09:40:09,949 - INFO - Dispensed 300 uL into well 3A1 with right pipette.


In [25]:
left_pipette.aspirate(
    volume=20,
    source=containers["7A1"],
    flow_rate=1
)

2025-03-24 09:29:54,732 - WARNING - Aspiration volume is larger than container volume!
2025-03-24 09:29:54,747 - INFO - Aspirated 20 uL from 7A1 with left pipette.


In [5]:
left_pipette.mixing(
    container=containers["3A1"],
    mix = ("after", 20, 5)
)

2025-03-24 09:40:36,149 - INFO - Done with mixing in 3A1 with order after, with volume 20 uL, repeated 5 times


In [24]:
left_pipette.volume = 0

In [6]:
right_pipette.aspirate(
    volume=300,
    source=containers["3A1"],
    touch_tip=True,
)
right_pipette.dispense(
    volume=300,
    destination=containers[well_id_trash],
    update_info=False
)

2025-03-24 09:41:17,332 - INFO - Touched tip performed, repeated 1 times
2025-03-24 09:41:17,332 - INFO - Container: aspirated 300 uL from this container with content pure mM water.
2025-03-24 09:41:17,354 - INFO - Aspirated 300 uL from 3A1 with right pipette.
2025-03-24 09:41:22,932 - INFO - Dispensed 300 uL into well 6A2 with right pipette.


In [7]:
right_pipette.drop_tip()

2025-03-24 09:45:05,750 - INFO - Right pipette dropped tip into trash.


In [8]:
left_pipette.clean_on_sponge()

2025-03-24 09:45:13,999 - INFO - Tip/needle cleaned on sponge.


In [9]:
left_pipette.return_needle()

2025-03-24 09:50:22,504 - INFO - Returned needle.


In [22]:
droplet_manager.measure_pendant_drop(
    source=containers["7A5"],
    drop_parameters={"drop_volume": 6, "max_measure_time": 60, "flow_rate": 1}
)

2025-03-21 14:52:25,911 - INFO - Start measurment of pendant drop of 7A5 with drop volume 6.0 uL and drop count 1.
2025-03-21 14:52:28,774 - INFO - Container: aspirated 17 uL from this container with content n.a. mM empty.
2025-03-21 14:52:28,774 - INFO - Aspirated 17 uL from 7A5 with left pipette.
2025-03-21 14:52:32,724 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-21 14:52:36,796 - INFO - Tip/needle cleaned on sponge.
2025-03-21 14:52:41,724 - INFO - Air gap of 3 uL removed in left pipette.
2025-03-21 14:52:41,741 - INFO - Dispensing pendant drop.
2025-03-21 14:52:48,726 - INFO - Camera: start measuring 7A5.
2025-03-21 14:52:50,875 - INFO - Camera: stopped measurement
2025-03-21 14:52:51,609 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-21 14:52:55,073 - INFO - Container: dispensed 17 uL into this container from source 7A5 containing n.a. mM empty.
2025-03-21 14:52:55,075 - INFO - Dispensed 17 uL into well 7A5 with left pipette.
2025-03-21 14:52:55,077 -

([],
 {'drop_volume': 6.0, 'max_measure_time': 60, 'flow_rate': 1, 'drop_count': 2})